## What we need 

Get data from WRDS --> is this through openAP

get data from FRED? -- Where is this? 
get fama french factors? 

How do we get the long-short positions?? like how do we see the returns?

are we able to get wrds connection or should we get a file downloaded from you 

will we be able to use gridsearchCV to run multiple models at once?

In [1]:
DEV = True # will shrink datasets after loading


In [2]:
import pandas as pd
import polars as pl

import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
crsp = pd.read_csv('crsp_data.csv')
crsp.info()

# for speedy dev, pick 20 permnos at random (with seed)
if DEV:
    np.random.seed(123)
    permnos = crsp['permno'].unique()
    permnos = np.random.choice(permnos, size=20, replace=False)
    crsp = crsp[crsp['permno'].isin(permnos)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4047630 entries, 0 to 4047629
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   permno  int64  
 1   date    object 
 2   ret     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 92.6+ MB


In [4]:
# crsp = pl.from_pandas(crsp)

crsp['date'] = pd.to_datetime(crsp['date'])

cleancrsp = crsp.query('ret.notnull() and date.dt.year >= 1975')
#crsp["ret"].is_not_null(), crsp['date'].dt.year() >= 1975) 
# keep only the data that has return and after 1975

len(cleancrsp['ret'].notnull()) # check that no null values in ret
len(cleancrsp['ret'].notnull()) == len(cleancrsp) # check that all ret values are not null

cleancrsp


,permno,date,ret
29180,10181,1986-05-30,6.8493
29181,10181,1986-06-30,16.6667
29182,10181,1986-07-31,1.0989
29183,10181,1986-08-29,-6.5217
29184,10181,1986-09-30,-4.6512
...,...,...,...
4037917,93235,2014-04-30,17.2319
4037918,93235,2014-05-30,-1.3066
4037919,93235,2014-06-30,5.7683
4037920,93235,2014-07-31,-2.9951


In [5]:
max_idx = cleancrsp['ret'].argmax()

cleancrsp.iloc[max_idx-5:max_idx+5] # this seems like an outlier for sure, not to be included in the data --> company is black stone minerals. 

,permno,date,ret
1789840,42076,1975-06-30,0.0000
1789841,42076,1975-07-31,8.3333
1789842,42076,1975-08-29,0.0000
1789843,42076,1975-09-30,-15.3846
1789844,42076,1975-10-31,-9.0909
2352007,61241,1975-01-31,147.0588
2352008,61241,1975-02-28,21.4286
2352009,61241,1975-03-31,33.3333
2352010,61241,1975-04-30,2.9412
2352011,61241,1975-05-30,31.4286


In [6]:
cleancrsp.sort_values('ret').tail(100) # check the top 10 returns
# these seem way too high

,permno,date,ret
2557794,69060,1983-06-30,30.4348
2352166,61241,1988-04-29,30.6818
2998916,78837,2000-03-31,30.6818
1229689,24482,2024-11-29,30.7692
2557777,69060,1982-01-29,30.7692
...,...,...,...
2557910,69060,1993-02-26,100.0000
2557783,69060,1982-07-30,100.0000
2557889,69060,1991-05-31,113.0435
1789836,42076,1975-02-28,114.2857


In [7]:
cleancrsp['ret'].describe(percentiles=[.01, .05, .10, .25, .5, .75, .9, .95, .99, .995]) # check the distribution of returns

count    2497.000000
mean        1.613311
std        15.738224
min       -66.573800
1%        -36.856788
5%        -21.481520
10%       -14.941300
25%        -6.060600
50%         0.539100
75%         7.337300
90%        17.835280
95%        26.547200
99%        53.333592
99.5%      63.974036
max       147.058800
Name: ret, dtype: float64

In [8]:
cleancrsp['permno'].nunique() # check the number of unique permnos



# profile = ProfileReport(cleancrsp, title="CRSP Data Profiling Report")


18

In [9]:
# profile

## Open Signals from Zip

In [ ]:
from zipfile import ZipFile


def open_dataframe_from_zip(zip_filepath, csv_filename):
    """
    Opens a Pandas DataFrame from a CSV file within a zip archive.

    Args:
        zip_filepath (str): Path to the zip file.
        csv_filename (str): Name of the CSV file inside the zip archive.

    Returns:
        pandas.DataFrame: The DataFrame read from the CSV file, or None if an error occurs.
    """
    try:
        with ZipFile(zip_filepath) as zf:
            with zf.open(csv_filename) as csvf:
                df = pd.read_csv(csvf)
        return df
    except FileNotFoundError:
        print(f"Error: Zip file not found at '{zip_filepath}'")
        return None
    except KeyError:
        print(f"Error: CSV file '{csv_filename}' not found in the zip archive.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage:
if DEV == False:
    zip_filepath = 'signed_predictors_dl_wide.zip'
    csv_filename = 'signed_predictors_dl_wide.csv'
    signals = open_dataframe_from_zip(zip_filepath, csv_filename)


    
# for speedy dev,
if DEV: 
    zip_filepath = 'signals.zip'
    csv_filename = 'signals.csv'
    signals = open_dataframe_from_zip(zip_filepath, csv_filename).reset_index().drop(columns=['Unnamed: 0'], axis=1) # drop the unnamed column and date column
    # signals = signals.iloc[:,:20] # only take the first 20 columns for dev

if signals is not None:
    print(signals.head())

   Unnamed: 0  permno        date  yyyymm_signals  AM  AOP  AbnormalAccruals  \
0           0   10000  1986-02-28          198601 NaN  NaN               NaN   
1           1   10000  1986-03-28          198602 NaN  NaN               NaN   
2           2   10000  1986-04-28          198603 NaN  NaN               NaN   
3           3   10000  1986-05-28          198604 NaN  NaN               NaN   
4           4   10000  1986-06-28          198605 NaN  NaN               NaN   

   Accruals  AccrualsBM  Activism1  ...  AgeIPO  AnalystRevision  \
0       NaN         NaN        NaN  ...     NaN              NaN   
1       NaN         NaN        NaN  ...     NaN              NaN   
2       NaN         NaN        NaN  ...     NaN              NaN   
3       NaN         NaN        NaN  ...     NaN              NaN   
4       NaN         NaN        NaN  ...     NaN              NaN   

   AnalystValue  AnnouncementReturn  AssetGrowth  BM  BMdec  BPEBM  Beta  \
0           NaN                 Na

In [11]:
signals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5273600 entries, 0 to 5273599
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   permno              int64  
 2   date                object 
 3   yyyymm_signals      int64  
 4   AM                  float64
 5   AOP                 float64
 6   AbnormalAccruals    float64
 7   Accruals            float64
 8   AccrualsBM          float64
 9   Activism1           float64
 10  Activism2           float64
 11  AdExp               float64
 12  AgeIPO              float64
 13  AnalystRevision     float64
 14  AnalystValue        float64
 15  AnnouncementReturn  float64
 16  AssetGrowth         float64
 17  BM                  float64
 18  BMdec               float64
 19  BPEBM               float64
 20  Beta                float64
 21  BetaFP              float64
dtypes: float64(18), int64(3), object(1)
memory usage: 885.2+ MB


In [12]:


signals.to_csv('signals.csv')

## want to make this a zip file to push
with ZipFile('signals.zip', 'w') as zf:
    zf.write('signals.csv')

In [13]:
def move_column_to_front(df, columns):
    return df[columns + [col for col in df.columns if col not in columns]]  




In [14]:

signals = signals.rename(columns={'yyyymm': 'yyyymm_signals'})


    # Create date that is one month ahead for merging with returns
signals["yyyymm_signals"] = signals["yyyymm_signals"].astype(str) 
signals["date"] = pd.to_datetime(signals["yyyymm_signals"] + "28", format="%Y%m%d") + pd.DateOffset(months=1)




In [15]:
signals = move_column_to_front(signals, ['permno', 'date']) # lag signals by a month

signals.head()

,permno,date,Unnamed: 0,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,...,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP
0,10000,1986-02-28,0,198601,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,1986-03-28,1,198602,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000,1986-04-28,2,198603,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000,1986-05-28,3,198604,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000,1986-06-28,4,198605,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:


# create a new datetime column, use the year and month, but make the date 28th of the month:
cleancrsp['date'] = pd.to_datetime(cleancrsp['date'].dt.year*100 + cleancrsp['date'].dt.month, format='%Y%m') + pd.DateOffset(days=27)


/var/folders/2m/j1mxfjpd7pl5zmd8sqqqcn_m0000gn/T/ipykernel_38013/2183361064.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleancrsp['date'] = pd.to_datetime(cleancrsp['date'].dt.year*100 + cleancrsp['date'].dt.month, format='%Y%m') + pd.DateOffset(days=27)


## Need to Convert yyyymm column to yyyy-mm-dd datetime column, rename to date

Something interesting, the crsp dataset uses trading days. How do we get this in our signals dataset. Just go off year-months?

Can we merge on yyyy-mm and not include dd so that we can match the crsp dates? But how would it know the date in the future? For instance, would it know the end of the month a year from now, or guess based on the previous crsp dd for that year or month before?

Is lagging the signals the right thing to do or do I lag the returns??


value weighting 


In [17]:
bigdata = cleancrsp.merge(signals, on=['permno', 'date'], how='left', indicator=True, validate="one_to_one") 
bigdata

,permno,date,ret,Unnamed: 0,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,...,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,_merge
0,10181,1986-05-28,6.8493,32991.0,198604,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,10181,1986-06-28,16.6667,32992.0,198605,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,10181,1986-07-28,1.0989,32993.0,198606,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,10181,1986-08-28,-6.5217,32994.0,198607,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,10181,1986-09-28,-4.6512,32995.0,198608,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,93235,2014-04-28,17.2319,5250538.0,201403,5.169856,NaN,NaN,-0.016947,NaN,...,NaN,NaN,-0.027726,0.026716,-0.046185,0.942494,-0.000611,0.087931,0.144612,both
2493,93235,2014-05-28,-1.3066,5250539.0,201404,4.409937,NaN,NaN,-0.016947,NaN,...,NaN,NaN,-0.027726,0.026716,-0.046185,0.942494,-0.002689,0.029387,0.193669,both
2494,93235,2014-06-28,5.7683,5250540.0,201405,4.468319,NaN,NaN,-0.016947,NaN,...,NaN,NaN,0.008933,0.026716,-0.046185,0.942494,-0.002556,0.031067,0.210866,both
2495,93235,2014-07-28,-2.9951,5250541.0,201406,4.224629,NaN,NaN,-0.016947,NaN,...,NaN,NaN,NaN,0.026716,-0.046185,0.970233,-0.003080,0.042468,0.240028,both


In [18]:
signal_list = signals.columns[3:] # get the list of signals
signals[signal_list].describe()

,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP
count,2.986693e+06,1.215192e+06,2.521764e+06,3.201801e+06,216647.000000,108731.000000,30170.000000,1.027386e+06,357438.000000,1.876115e+06,1.215192e+06,2.861710e+06,3.237201e+06,2.676641e+06,2.940113e+06,2.873411e+06,4.187358e+06,3.709235e+06
mean,3.681935e+00,-1.615377e+02,8.002906e-05,3.121143e-02,0.483616,14.886500,9.263089,7.553823e-02,17.272352,1.008518e+00,7.514384e-01,2.473509e-03,-1.736418e-01,-7.312798e-01,2.942087e+00,-3.756112e-02,9.894609e-01,9.732591e-01
std,2.688265e+01,4.798586e+04,1.551799e-01,1.406179e-01,0.499733,2.724359,12.642133,3.686100e-01,22.451399,4.864047e+00,1.553453e+00,1.013126e-01,2.696810e+00,1.041276e+00,5.282437e+01,3.385324e+02,7.430238e-01,6.358664e-01
min,0.000000e+00,-1.526718e+07,-2.815177e+00,-1.161905e+01,0.000000,6.000000,0.000000,8.399294e-07,0.000000,-1.046000e+03,-4.581968e+01,-1.608569e+00,-1.071381e+03,-1.013099e+01,-4.881022e+03,-1.210335e+05,-1.786632e+01,3.799159e-07
25%,6.427771e-01,-1.356250e+00,-5.234549e-02,-1.200336e-02,0.000000,13.000000,0.000000,4.923979e-03,5.000000,9.914712e-01,4.473222e-01,-3.780500e-02,-1.875365e-01,-1.281019e+00,3.623412e-01,-2.340956e-01,5.297898e-01,5.146013e-01
50%,1.384538e+00,-3.967994e-01,-6.826031e-03,2.897744e-02,0.000000,15.000000,7.438817,1.628784e-02,9.000000,1.000000e+00,6.884428e-01,0.000000e+00,-6.485017e-02,-5.981418e-01,6.817607e-01,-1.626706e-02,8.991139e-01,8.896496e-01
75%,3.113225e+00,-6.328331e-02,4.068833e-02,7.237209e-02,1.000000,17.000000,10.728353,5.602462e-02,18.000000,1.006897e+00,9.908170e-01,3.922600e-02,2.697890e-02,-4.733975e-02,1.168479e+00,6.726695e-02,1.330791e+00,1.309197e+00
max,1.230961e+04,2.354832e+04,7.641108e+00,2.431375e+01,1.000000,23.000000,221.282650,9.411890e+01,227.000000,5.783554e+03,3.933582e+02,9.453482e+00,1.000000e+00,4.398199e+00,1.396147e+04,4.717321e+05,5.263980e+01,1.261470e+01


In [19]:
# pretend its a dataframe grab the rows where the column is the name and make that a list

signal_documentation = pd.read_csv('SignalDoc.csv')
signal_documentation

# need to find a way to add the column type from cat. form to match the acronyms which are the columns of the dataset
signaldoc = signal_documentation[['Acronym', 'Cat.Form']].drop_duplicates()


In [20]:

from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline 
from sklearn.impute import SimpleImputer
from df_after_transform import df_after_transform
from sklearn.model_selection import KFold, cross_validate, GridSearchCV

In [ ]:
## First We need to split the data into train and test sets
## Don't use trqin, test, split

In [ ]:
from sklearn.linear_model import Ridge

continuous_cols = signaldoc.loc[signaldoc['Cat.Form'] == 'continuous']
discrete_cols = signaldoc.loc[signaldoc['Cat.Form'] == 'discrete']

numer_pipe = make_pipeline(SimpleImputer(strategy='mean'),
                           StandardScaler()) 

cat_pipe   = make_pipeline(OneHotEncoder(drop='first'))


preproc_pipe = ColumnTransformer(
    transformers=[
        ('numbers', numer_pipe, continuous_cols),  
        ('categoricals', cat_pipe, discrete_cols), 
    ],
    remainder='drop'  
)

model_pipe = make_pipeline(
    preproc_pipe,
    Ridge()
)

# model_pipe.fit(, )


NameError: name 'X_train' is not defined